In [ ]:
# run against production 2024-11-05
from tqdm import tqdm
from dumper.utils import gsheet_to_df
from django.core.exceptions import ObjectDoesNotExist
from django.db import IntegrityError
from apis_core.utils import get_object_from_pk_or_uri
from AcdhArcheAssets.uri_norm_rules import get_normalized_uri
import pandas as pd

In [ ]:
col, _ = Collection.objects.get_or_create(name="Schnitzler-Kino")

In [ ]:
df = gsheet_to_df("1-Kl0pFsiABqzNZ65PoGp2SWrGadcRega-DIs_NxLcHE")

In [ ]:
df.head()

In [ ]:
for i, row in tqdm(df.iterrows(), total=len(df)):
    film_id = row["film-id"]
    try:
        item = Work.objects.get(id=film_id)
    except ObjectDoesNotExist:
        print(film_id)
    name = row["titel-korrektur"]
    if isinstance(name, str):
        item.name = name
    try:
        start_date = str(int(row["date-korrektur"]))
        item.start_date_written = start_date
        item.end_date_written = start_date
    except ValueError:
        pass
    if isinstance(row["wikidata-neu"], str):
        url = get_normalized_uri(row["wikidata-neu"])
        try:
            uri = Uri.objects.get(
                uri=url
            )
        except ObjectDoesNotExist:
            uri = Uri.objects.create(
                uri=url,
                domain="wikidata",
            )
        uri.entity = item
        uri.save()
    if isinstance(row["imdb-neu"], str):
        url = get_normalized_uri(row["imdb-neu"])
        try:
            uri = Uri.objects.get(
                uri=url
            )
        except ObjectDoesNotExist:
            uri = Uri.objects.create(
                uri=url,
                domain="imdb",
            )
        uri.entity = item
        uri.save()
        
    item.save()